In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import split, col, to_date, regexp_replace

# Caminho para o banco de dados SQLite
db_path = "/home/sdna/Documents/python_portal_transparencia/portal.db"
jdbc_url = f"jdbc:sqlite:{db_path}"

# Nome da tabela no banco de dados SQLite
#precisei copiar o arquivo jar jdbc para a pasta spark\jars 

# Inicialize a SparkSession
spark = SparkSession.builder \
    .appName("PySpark com SQLite via JDBC") \
    .config("spark.jars.packages", "org.xerial:sqlite-jdbc:3.41.2.1") \
    .getOrCreate()

# Leia os dados da tabela do SQLite usando JDBC
try:
    df = spark.read \
        .format("jdbc") \
        .option("url", jdbc_url) \
        .option("dbtable", "DT_PORTAL_BRONZE") \
        .option("driver", "org.sqlite.JDBC") \
        .load()
    
    #Transformacao variaveis
    df = df.withColumn('SALARIO', regexp_replace('SALARIO', '\.', '')) \
        .withColumn('SALARIO', regexp_replace('SALARIO', 'R\$.', '')) \
            .withColumn('SALARIO', regexp_replace('SALARIO', ',', '\.')) 
 
    df = df.withColumn("SALARIO",df.SALARIO.cast('double')) \
        .withColumn("DATA_CONTRATACAO", to_date(df["DATA_CONTRATACAO"], "dd/MM/yyyy"))   

    df.write \
        .format("jdbc") \
        .option("url", jdbc_url) \
        .option("dbtable", "DT_PORTAL_PRATA") \
        .option("driver", "org.sqlite.JDBC") \
        .mode("append") \
        .save()        

    # Mostre os dados lidos
    df.show()
except Exception as e:
    print("Erro ao ler os dados com JDBC:", e)
finally:
    # Feche a SparkSession
    spark.stop()


24/08/18 15:36:03 WARN JdbcUtils: Requested isolation level 1 is not supported; falling back to default isolation level 8


+---+--------------------+--------------------+-------+------------+----------------+-------------+
| id|                NOME|               CARGO|SALARIO|TIPO_VINCULO|DATA_CONTRATACAO|DATA_DEMISSAO|
+---+--------------------+--------------------+-------+------------+----------------+-------------+
|  1|LARA ALMEIDA DE O...|            ADVOGADO| 1500.0|    Contrato|      2022-02-16|             |
|  2|AGNALDO PINHEIRO ...| AG GESTAO ADMINISTR| 1412.0|     Efetivo|      2022-05-09|             |
|  3|BARBARA KELLY FER...| AG GESTAO ADMINISTR| 1412.0|     Efetivo|      2022-05-09|             |
|  4|ERLANE DIAS DA SILVA| AG GESTAO ADMINISTR| 1412.0|     Efetivo|      2022-05-09|             |
|  5|LUCIANE FELIX DE ...| AG GESTAO ADMINISTR| 1412.0|     Efetivo|      2022-05-09|             |
|  6|MARIA DOS ANJOS T...| AG GESTAO ADMINISTR| 1412.0|     Efetivo|      2022-05-09|             |
|  7|NILMARA FREITAS S...| AG GESTAO ADMINISTR| 1412.0|     Efetivo|      2022-05-09|             |
